In [ ]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.9 MB/s eta 0:00:00


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언

인자(argument)의 역할과 내용
- pretrained_model_name : 이전 장에서 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델 이름(단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)
- downstream_model_dir : 이전 장에서 파인튜닝한 모델의 체크포인트 저장 위치.
- max_seq_length : 토큰 기준 입력 문장 최대 길이. 아무 것도 입력하지 않으면 128입니다.

In [ ]:
# 인퍼런스 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls1",
    max_seq_length=128,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls1/epoch=0-val_loss=0.27-v2.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

- fine_tuned_model_ckpt['state_dict']: 파인튜닝된 모델의 상태를 가지고 있는 딕셔너리. 모델의 각 레이어에 대한 가중치와 bias 값들 저장
- 'model.classifier.bias': 분류기의 bias 텐서를 가져온다.이 분류기는 각 클래스에 대한 확률을 출력하며, bias 텐서 크기는 분류 클래스 수와 동일
- .shape.numel(): 텐서의 모든 원소의 개수를 반환. 이 경우에는 bias 텐서의 원소의 개수, 즉 클래스의 수를 반환


In [ ]:
import torch
from transformers import BertConfig, BertForSequenceClassification
# 체크포인트 로드
# downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls1/epoch=0-val_loss=0.27-v1.ckpt
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu") # 체크포인트를 CPU 메모리에 로드
)
# 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델의 설정 값들을 읽어 들일 수 있음
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
# 초기화한 BERT 모델에 체크포인트(fine_tuned_model_ckpt)를 읽어들이게 된다
model = BertForSequenceClassification(pretrained_model_config) # birt 모델 초기화
# "model.layer_name" 형태의 체크포인트 키 이름을 "layer_name" 형태로 바꾼 후, 이를 현재 모델에 주입.
# 이를 통해 파인튜닝된 모델의 가중치를 현재 모델에 사용
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()}) # 체크포인트 주입
# 모델이 평가 모드로 전환. 드롭아웃 등 학습 때만 사용하는 기법들을 무효화하는 역할을 한다.
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# 토크나이저 초기화
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 인퍼런스 함수 선언
- 문장(sentence)에 토큰화를 수행한 뒤 input_ids, attention_mask, token_type_ids를 만든다.
- 이들 입력값을 파이토치 텐서(tensor) 자료형으로 변환한 뒤 모델에 입력합니다. - 모델 출력 값(outputs.logits)은 소프트맥스 함수 적용 이전의 로짓(logit) 형태인데요. 여기에 소프트맥스 함수를 써서 모델 출력을 [부정일 확률, 긍정일 확률] 형태의 확률 형태로 바꾼다.
- 마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 부정 위치일 경우 해당 문장이 부정(positive), 반대의 경우 긍정(positive)이 되도록 pred 값을 만든다.

In [ ]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{round(negative_prob * 100,2)}%",
    }

In [ ]:
sentence = '갈까 말까'
inference_fn(sentence)

{'sentence': '갈까 말까',
 'prediction': '부정 (negative)',
 'positive_data': '긍정 0.3484',
 'negative_data': '부정 0.6516',
 'positive_width': '34.839999999999996%',
 'negative_width': '65.16%'}

In [ ]:
sentence = '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??'
inference_fn(sentence)

{'sentence': '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??',
 'prediction': '부정 (negative)',
 'positive_data': '긍정 0.0691',
 'negative_data': '부정 0.9309',
 'positive_width': '6.909999999999999%',
 'negative_width': '93.09%'}

In [ ]:
!cp -r /gdrive/MyDrive/nlpbook/checkpoint-doccls1 /gdrive/MyDrive/hmkd1_share

In [ ]:
!rm -r /gdrive/MyDrive/nlpbook/checkpoint-doccls1g

## ngrok
ngrok 사이트에는 Secure tunnels to localhost이라고 설명되어 있습니다. 간단히 방화벽 넘어서 외부에서 로컬에 접속 가능하게 하는 터널 프로그램입니다. 만약 통신을 할 수 있는 프로그램을 개발했다면 서버의 역할이 필요한데 개발한 PC의 경우는 외부에서 접속가능한 상용 서버가 아니기때문에 도메인을 구입하고 서버를 호스팅 받아 연결해야 합니다. 말그대로 로컬서버죠.

ngrok은 이러한 로컬서버를 간단히 외부에서 접속가능한 환경으로 만들 수 있게 도와줍니다. 물론 이런 프로그램이 ngrok뿐인 것은 아니지만, 위에서 설명한 상황에서 유용하게 쓸 수 있게 최적화되어 있어서 아주 쉽게 사용할 수 있다.

## ngrok 설치
ngrok 사이트로 이동해서 [Get started for free]를 눌러 가입을 합니다. 이미 가입이 된 상태라면 Login을 하시면 됩니다.

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [ ]:
!mkdir /root/.ngrok2 && echo "" > /root/.ngrok2/ngrok.yml

In [ ]:
!ls /root -al

total 72
drwx------ 1 root root 4096 Jul 26 01:48 .
drwxr-xr-x 1 root root 4096 Jul 26 00:45 ..
-r-xr-xr-x 1 root root 1169 Jan  1  2000 .bashrc
drwxr-xr-x 1 root root 4096 Jul 26 00:47 .cache
drwx------ 1 root root 4096 Jul 26 00:45 .config
drwxr-xr-x 5 root root 4096 Jul 20 13:44 .ipython
drwx------ 1 root root 4096 Jul 20 13:44 .jupyter
drwxr-xr-x 2 root root 4096 Jul 20 13:42 .keras
drwx------ 3 root root 4096 Jul 20 13:21 .launchpadlib
drwxr-xr-x 1 root root 4096 Jul 20 13:44 .local
drwxr-xr-x 2 root root 4096 Jul 26 01:48 .ngrok2
drwxr-xr-x 4 root root 4096 Jul 20 13:44 .npm
-rw-r--r-- 1 root root  161 Jul  9  2019 .profile
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf
-rw-r--r-- 1 root root  165 Jul 20 13:44 .wget-hsts


# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app 'ratsnlp.nlpbook.classification.deploy'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://546e-34-74-55-119.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:14:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:14:04] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:14:54] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:15:17] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:15:38] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:15:57] "POST /api HTTP/1.1" 200 -
